# Clustering Crypto

In [23]:
# Initial imports
import pandas as pd
import hvplot.pandas
from bokeh.sampledata.autompg import autompg_clean as df
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [2]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col =0)
crypto_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

In [3]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

In [4]:
# Keep all the cryptocurrencies that have a working algorithm.
# YOUR CODE HERE

In [5]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=['IsTrading'])
crypto_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
365    365Coin       X11   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       PoW              NaN          611000
808        808   SHA-256   PoW/PoS     0.000000e+00               0

In [6]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
crypto_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404     404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
808         808   SHA-256   PoW/PoS     0.000000e+00               0
1337  EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC     Bitcoin   SHA-256       PoW     1.792718e+07        21000000

In [7]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404     404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
1337  EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC     Bitcoin   SHA-256       PoW     1.792718e+07        21000000
ETH    Ethereum    Ethash       PoW     1.076842e+08               0

In [8]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name_df = crypto_df[["CoinName"]]
coin_name_df.head()

CoinName
42      42 Coin
404     404Coin
1337  EliteCoin
BTC     Bitcoin
ETH    Ethereum

In [9]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns=['CoinName'])
crypto_df.head(10)

Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42            Scrypt   PoW/PoS     4.199995e+01              42
404           Scrypt   PoW/PoS     1.055185e+09       532000000
1337             X13   PoW/PoS     2.927942e+10    314159265359
BTC          SHA-256       PoW     1.792718e+07        21000000
ETH           Ethash       PoW     1.076842e+08               0
LTC           Scrypt       PoW     6.303924e+07        84000000
DASH             X11   PoW/PoS     9.031294e+06        22000000
XMR   CryptoNight-V7       PoW     1.720114e+07               0
ETC           Ethash       PoW     1.133597e+08       210000000
ZEC         Equihash       PoW     7.383056e+06        21000000

In [10]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm","ProofType"])
X

TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
42       4.199995e+01              42                                 0   
404      1.055185e+09       532000000                                 0   
1337     2.927942e+10    314159265359                                 0   
BTC      1.792718e+07        21000000                                 0   
ETH      1.076842e+08               0                                 0   
...               ...             ...                               ...   
ZEPH     2.000000e+09      2000000000                                 0   
GAP      1.493105e+07       250000000                                 0   
BDX      9.802226e+08      1400222610                                 0   
ZEN      7.296538e+06        21000000                                 0   
XBC      1.283270e+05         1000000                                 0   

      Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  Algorithm_Blake  \
42                0                  0                   0                0   
404               0                  0                   0                0   
1337              0                  0                   0                0   
BTC               0                  0                   0                0   
ETH               0                  0                   0                0   
...             ...                ...                 ...              ...   
ZEPH              0                  0                   0                0   
GAP               0                  0                   0                0   
BDX               0                  0                   0                0   
ZEN               0                  0                   0                0   
XBC               0                  0                   0                0   

      Algorithm_Blake2S  Algorithm_Blake2b  Algorithm_C11  ...  \
42                    0                  0              0  ...   
404                   0                  0              0  ...   
1337                  0                  0              0  ...   
BTC                   0                  0              0  ...   
ETH                   0                  0              0  ...   
...                 ...                ...            ...  ...   
ZEPH                  0                  0              0  ...   
GAP                   0                  0              0  ...   
BDX                   0                  0              0  ...   
ZEN                   0                  0              0  ...   
XBC                   0                  0              0  ...   

      ProofType_PoW/PoS  ProofType_PoW/PoS   ProofType_PoW/PoW  \
42                    1                   0                  0   
404                   1                   0                  0   
1337                  1                   0                  0   
BTC                   0                   0                  0   
ETH                   0                   0                  0   
...                 ...                 ...                ...   
ZEPH                  0                   0                  0   
GAP                   1                   0                  0   
BDX                   0                   0                  0   
ZEN                   0                   0                  0   
XBC                   0                   0                  0   

      ProofType_PoW/nPoS  ProofType_Pos  ProofType_Proof of Authority  \
42                     0              0                             0   
404                    0              0                             0   
1337                   0              0                             0   
BTC                    0              0                             0   
ETH                    0              0                             0   
...                  ...            ...                           ...   
ZEPH                   0              0                    

In [11]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(X)
print(crypto_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [12]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
print(crypto_pca)

[[-0.33841851  1.05610123 -0.55764287]
 [-0.32172517  1.05636126 -0.55806696]
 [ 2.30216221  1.63547745 -0.65853857]
 ...
 [ 0.3320043  -2.27736706  0.39079798]
 [-0.15424466 -2.07662144  0.57882662]
 [-0.28707534  0.91600337 -0.28237374]]


In [13]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data = crypto_pca, columns = ["PC1", "PC2", "PC3"], 
    index = crypto_df.index
)
pcs_df.head(10)

PC1       PC2       PC3
42   -0.338419  1.056101 -0.557643
404  -0.321725  1.056361 -0.558067
1337  2.302162  1.635477 -0.658539
BTC  -0.146657 -1.347616  0.230824
ETH  -0.148099 -2.027329  0.392103
LTC  -0.163100 -1.057736 -0.002942
DASH -0.397550  1.125048 -0.514759
XMR  -0.153427 -2.212033  0.372991
ETC  -0.146538 -2.027419  0.392081
ZEC  -0.154244 -2.076621  0.578827

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [14]:
# Create an elbow curve to find the best value for K.
#vars needed for elbow curve 
inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(X)
   inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [15]:
# Initialize the K-Means model.
def get_clusters(k, data):
   # Create a copy of the DataFrame
   data = data.copy()

   # Initialize the K-Means model
   model = KMeans(n_clusters=k, random_state=0)

   # Fit the model
   model.fit(data)

   # Predict clusters
   predictions = model.predict(data)

   # Create return DataFrame with predicted clusters
   data["class"] = model.labels_

   return data

In [16]:
pcs_df = get_clusters(4,pcs_df)
pcs_df.head()

PC1       PC2       PC3  class
42   -0.338419  1.056101 -0.557643      0
404  -0.321725  1.056361 -0.558067      0
1337  2.302162  1.635477 -0.658539      0
BTC  -0.146657 -1.347616  0.230824      1
ETH  -0.148099 -2.027329  0.392103      1

In [17]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df, coin_name_df], axis=1)


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


Algorithm ProofType  TotalCoinsMined TotalCoinSupply       PC1  \
42            Scrypt   PoW/PoS     4.199995e+01              42 -0.338419   
404           Scrypt   PoW/PoS     1.055185e+09       532000000 -0.321725   
1337             X13   PoW/PoS     2.927942e+10    314159265359  2.302162   
BTC          SHA-256       PoW     1.792718e+07        21000000 -0.146657   
ETH           Ethash       PoW     1.076842e+08               0 -0.148099   
LTC           Scrypt       PoW     6.303924e+07        84000000 -0.163100   
DASH             X11   PoW/PoS     9.031294e+06        22000000 -0.397550   
XMR   CryptoNight-V7       PoW     1.720114e+07               0 -0.153427   
ETC           Ethash       PoW     1.133597e+08       210000000 -0.146538   
ZEC         Equihash       PoW     7.383056e+06        21000000 -0.154244   

           PC2       PC3  class          CoinName  
42    1.056101 -0.557643      0           42 Coin  
404   1.056361 -0.558067      0           404Coin  
1337  1.635477 -0.658539      0         EliteCoin  
BTC  -1.347616  0.230824      1           Bitcoin  
ETH  -2.027329  0.392103      1          Ethereum  
LTC  -1.057736 -0.002942      1          Litecoin  
DASH  1.125048 -0.514759      0              Dash  
XMR  -2.212033  0.372991      1            Monero  
ETC  -2.027419  0.392081      1  Ethereum Classic  
ZEC  -2.076621  0.578827      1             ZCash

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [22]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [25]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply', 'TotalCoinsMined', 'class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [32]:
# Print the total number of tradable cryptocurrencies.
num = len(clustered_df.index)
# YOUR CODE HERE
print("There are " + f'{num}' + " tradable cryptocurrencies")

There are 532 tradable cryptocurrencies


In [37]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
#  MinMaxScaler().fit_transform
data = clustered_df[['TotalCoinSupply','TotalCoinsMined']]
mini_scaled_data = MinMaxScaler().fit_transform(data)
print (mini_scaled_data)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [45]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
    data = mini_scaled_data, 
    columns = ["TotalCoinSupply", "TotalCoinsMined"], 
    index = crypto_df.index
)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df = pd.concat([plot_df, coin_name_df], axis=1)

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
class_df = clustered_df['class']
plot_df= pd.concat([plot_df, class_df], axis=1)

plot_df.head(10)

TotalCoinSupply  TotalCoinsMined          CoinName  class
42       4.200000e-11         0.000000           42 Coin      0
404      5.320000e-04         0.001066           404Coin      0
1337     3.141593e-01         0.029576         EliteCoin      0
BTC      2.100000e-05         0.000018           Bitcoin      1
ETH      0.000000e+00         0.000109          Ethereum      1
LTC      8.400000e-05         0.000064          Litecoin      1
DASH     2.200000e-05         0.000009              Dash      0
XMR      0.000000e+00         0.000017            Monero      1
ETC      2.100000e-04         0.000115  Ethereum Classic      1
ZEC      2.100000e-05         0.000007             ZCash      1

In [46]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinSupply",
    y="TotalCoinsMined",
    hover_cols=["class"],
    by="class",
)


:NdOverlay   [class]
   :Scatter   [TotalCoinSupply]   (TotalCoinsMined,class)